In [1]:
!pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 7.0 MB/s 
     |████████████████████████████████| 162 kB 87.0 MB/s 
     |████████████████████████████████| 182 kB 66.0 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 162 kB 45.8 MB/s 
     |████████████████████████████████| 158 kB 24.2 MB/s 
     |████████████████████████████████| 157 kB 57.9 MB/s 
     |████████████████████████████████| 157 kB 48.8 MB/s 
     |████████████████████████████████| 157 kB 48.3 MB/s 
     |████████████████████████████████| 157 kB 52.2 MB/s 
     |████████████████████████████████| 157 kB 34.4 MB/s 
     |████████████████████████████████| 157 kB 49.1 MB/s 
     |████████████████████████████████| 157 kB 48.9 MB/s 
     |████████████████████████████████| 156 kB 55.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=e8

In [2]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
# import the necessary packages
from imutils import paths
import logging
import os
import cv2
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [4]:
# configure logging
# reference for a logging obj
logger = logging.getLogger()

# set level of logging
logger.setLevel(logging.INFO)

# create handlers
c_handler = logging.StreamHandler()
c_format = logging.Formatter(fmt="%(asctime)s %(message)s",datefmt='%d-%m-%Y %H:%M:%S')
c_handler.setFormatter(c_format)

# add handler to the logger
logger.handlers[0] = c_handler

In [5]:
# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
  "project_name": "second_image_classifier",
  "train_feature_artifact": "train_x:latest",
  "train_target_artifact": "train_y:latest",
  "test_feature_artifact": "test_x:latest",
  "test_target_artifact": "test_y:latest",
  "encoder": "target_encoder",
  "inference_model": "model",
  "deploy": "inference_result"
}

In [9]:
sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
    'solver': {
        'values': ['adam', 'sgd']
        },
    'max_iter': {
        'values': [100, 200]
        },
    'activation': {
        'values': ['relu', 'tanh']
        },
     'hidden_layer_sizes': {
          'values': [(40,5), (40,10)]
        },
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, entity='gildson', project=args["project_name"])

Create sweep with ID: f12r803t
Sweep URL: https://wandb.ai/gildson/second_image_classifier/sweeps/f12r803t


In [10]:
def train():
  run = wandb.init(entity="gildson",project=args["project_name"], job_type="Train")
  logger.info("Downloading the training data")
  train_x_artifact = run.use_artifact(args["train_feature_artifact"])
  train_x_path = train_x_artifact.file()
  train_y_artifact = run.use_artifact(args["train_target_artifact"])
  train_y_path = train_y_artifact.file()

  # unpacking the artifacts
  train_x = joblib.load(train_x_path)
  train_y = joblib.load(train_y_path)
  # encode the labels as integers
  le = LabelEncoder()
  train_y = le.fit_transform(train_y)

  model = MLPClassifier(hidden_layer_sizes=wandb.config.hidden_layer_sizes,
                          max_iter=wandb.config.max_iter,
                          solver=wandb.config.solver,
                          activation=wandb.config.activation,
                          random_state=1,
                          verbose=True)
  logger.info("[INFO] training MLP classifier...")
  model.fit(train_x, train_y)

  logger.info("Dumping the model and encoder artifacts to the disk")

  # Save the artifacts using joblib
  joblib.dump(le, args["encoder"])
  joblib.dump(model, args["inference_model"])

In [11]:
wandb.agent(sweep_id, function=train, count = 10)

wandb: Agent Starting Run: 0lkiz27c with config:
wandb: 	activation: relu
wandb: 	hidden_layer_sizes: [40, 10]
wandb: 	max_iter: 100
wandb: 	solver: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


16-10-2022 21:36:59 Downloading the training data
16-10-2022 21:37:00 [INFO] training MLP classifier...


Iteration 1, loss = 4.75960843
Iteration 2, loss = 1.15258605
Iteration 3, loss = 1.14873495
Iteration 4, loss = 1.14474702
Iteration 5, loss = 1.14109281
Iteration 6, loss = 1.13760306
Iteration 7, loss = 1.13449398
Iteration 8, loss = 1.13158467
Iteration 9, loss = 1.12893401
Iteration 10, loss = 1.12643630
Iteration 11, loss = 1.12409272
Iteration 12, loss = 1.12202996
Iteration 13, loss = 1.12011877
Iteration 14, loss = 1.11841623
Iteration 15, loss = 1.11682307
Iteration 16, loss = 1.11536589
Iteration 17, loss = 1.11393733
Iteration 18, loss = 1.11276292
Iteration 19, loss = 1.11168284
Iteration 20, loss = 1.11063162
Iteration 21, loss = 1.10967813
Iteration 22, loss = 1.10877829
Iteration 23, loss = 1.10794667
Iteration 24, loss = 1.10720065
Iteration 25, loss = 1.10649099
Iteration 26, loss = 1.10587056
Iteration 27, loss = 1.10530499
Iteration 28, loss = 1.10484762
Iteration 29, loss = 1.10434948
Iteration 30, loss = 1.10391290
Iteration 31, loss = 1.10352156
Iteration 32, los

16-10-2022 21:37:07 Dumping the model and encoder artifacts to the disk


Iteration 59, loss = 1.09947449
Iteration 60, loss = 1.09945304
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


...

wandb: Agent Starting Run: nnb19inv with config:
wandb: 	activation: relu
wandb: 	hidden_layer_sizes: [40, 10]
wandb: 	max_iter: 100
wandb: 	solver: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


16-10-2022 21:40:16 Downloading the training data
16-10-2022 21:40:17 [INFO] training MLP classifier...


Iteration 1, loss = 4.75960843
Iteration 2, loss = 1.15258605
Iteration 3, loss = 1.14873495
Iteration 4, loss = 1.14474702
Iteration 5, loss = 1.14109281
Iteration 6, loss = 1.13760306
Iteration 7, loss = 1.13449398
Iteration 8, loss = 1.13158467
Iteration 9, loss = 1.12893401
Iteration 10, loss = 1.12643630
Iteration 11, loss = 1.12409272
Iteration 12, loss = 1.12202996
Iteration 13, loss = 1.12011877
Iteration 14, loss = 1.11841623
Iteration 15, loss = 1.11682307
Iteration 16, loss = 1.11536589
Iteration 17, loss = 1.11393733
Iteration 18, loss = 1.11276292
Iteration 19, loss = 1.11168284
Iteration 20, loss = 1.11063162
Iteration 21, loss = 1.10967813
Iteration 22, loss = 1.10877829
Iteration 23, loss = 1.10794667
Iteration 24, loss = 1.10720065
Iteration 25, loss = 1.10649099
Iteration 26, loss = 1.10587056
Iteration 27, loss = 1.10530499
Iteration 28, loss = 1.10484762
Iteration 29, loss = 1.10434948
Iteration 30, loss = 1.10391290
Iteration 31, loss = 1.10352156
Iteration 32, los

16-10-2022 21:40:24 Dumping the model and encoder artifacts to the disk


Iteration 59, loss = 1.09947449
Iteration 60, loss = 1.09945304
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


In [12]:
run = wandb.init(entity="gildson",project=args["project_name"], job_type="Train")

# encoder artifact
artifact = wandb.Artifact(args["encoder"],
                          type="INFERENCE_MODEL",
                          description="A json file representing the target encoder"
                          )

logger.info("Logging the target encoder artifact")
artifact.add_file(args["encoder"])
run.log_artifact(artifact)

# inference model artifact
artifact = wandb.Artifact(args["inference_model"],
                          type="INFERENCE_MODEL",
                          description="A json file representing the inference model"
                          )

logger.info("Logging the inference model artifact")
artifact.add_file(args["inference_model"])
run.log_artifact(artifact)

run.finish()

wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


16-10-2022 21:40:37 Logging the target encoder artifact
16-10-2022 21:40:37 Logging the inference model artifact
